In [1]:
import sys
sys.path.append('..')  # Add parent directory to path
from utils.backtest import Backtest
from utils.data_handler import DataHandler

from strategies.ratio_value import RatioValueStrategy
from strategies.momentum import MomentumStrategy
from utils.backtest import Backtest
from utils.data_handler import DataHandler
import yfinance as yf
import pandas as pd

In [2]:
import pandas as pd

# Define tickers and parameter grid
tickers = [
    "AAPL", "ABT", "ADI", "ADM", "ADP", "ADSK", "AEE", "AEP", "AJG", "AKAM",
    "ALB", "AMAT", "AMD", "AME", "AMGN", "AMT", "AMZN", "AON", "AOS", "APA",
    "APD", "APH", "ARE", "AVB", "AVY", "AXP", "AZO", "BA", "BAC", "BAX",
    "BBY", "BDX", "BEN", "BIIB", "BK", "BLK", "BMY", "BSX", "BXP", "C",
    "CAG", "CAH", "CAT", "CB", "CCI", "CDNS", "CHD", "CHRW", "CINF", "CL",
    "CLX", "CMI", "CNP", "COO", "COP", "COST", "CPB", "CPRT", "CSCO", "CTAS",
    "CTSH", "D", "DE", "DGX", "DHI", "DIS", "DLTR", "DOV", "DRI", "DTE",
    "DVA", "EA", "EBAY", "ECL", "ED", "EFX", "EIX", "EL", "EMN", "EMR",
    "EOG", "EQR", "ES", "ESS", "F", "FAST", "FCX", "FDX", "FE", "FFIV",
    "FITB", "FRT", "GD", "GE", "GILD", "GIS", "GPC", "GWW", "HAL", "HAS"
] # Add more tickers as needed
frequencies = ['weekly', 'monthly', '6month']
K = [0.5, 1, 1.5]

# Initialize final results container
all_results = {}
best_params = []

# Loop over each ticker
for ticker in tickers:
    print(f"\n--- Evaluating {ticker} ---")

    # Initialize data
    data_handler = DataHandler(ticker=ticker, start_date='2004-12-12', end_date='2015-12-12')
    prices = data_handler.fetch_data()
    pb_series = data_handler.fetch_pe_series('../data/PE RATIO.xlsx')

    results = {}

    # Loop over each frequency and threshold
    for freq in frequencies:
        results[freq] = {}
        for thresh in K:
            print(f"\nTesting {freq} frequency strategy with k={thresh} for {ticker}:")

            # Initialize strategy
            strategy = RatioValueStrategy(pb_series, "PE", window=5, k=thresh, frequency=freq)

            # Run backtest
            backtest = Backtest(
                data=prices,
                strategy=strategy,
                plot_results=False
            )

            # Store results
            result = backtest.run()
            results[freq][thresh] = result

            # Print metrics
            print(f"Final Portfolio Value: ${result['Final Value']:,.2f}")
            print(f"Total Return: {result['Return']:.2%}")
            print(f"Sharpe Ratio: {result['Sharpe Ratio']:.2f}")
            print(f"Max Drawdown: {result['Max Drawdown']:.2%}")

    # Save all results for this ticker
    all_results[ticker] = results

    # Flatten and create comparison DataFrame
    comparison = pd.concat({
        freq: pd.DataFrame.from_dict(results[freq], orient='index')
        for freq in frequencies
    }, names=['Frequency', 'k'])

    # Find best combo based on Sharpe Ratio
    best_idx = comparison['Sharpe Ratio'].idxmax()
    best_row = comparison.loc[best_idx]
    best_params.append({
        'Ticker': ticker,
        'Best Frequency': best_idx[0],
        'Best k': best_idx[1],
        'Sharpe Ratio': best_row['Sharpe Ratio'],
        'Total Return': best_row['Return'],
        'Final Value': best_row['Final Value'],
        'Max Drawdown': best_row['Max Drawdown']
    })

    # Optionally show per-stock comparison table
    print("\nStrategy Comparison Table:")
    print(comparison.round(4))

# Convert best parameter summary to DataFrame
best_params_df = pd.DataFrame(best_params)
print("\n=== Summary of Best Parameters by Ticker ===")
print(best_params_df.round(4))



--- Evaluating AAPL ---
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed



Testing weekly frequency strategy with k=0.5 for AAPL:
Final Portfolio Value: $136,694.83
Total Return: 36.69%
Sharpe Ratio: 0.24
Max Drawdown: -65.41%

Testing weekly frequency strategy with k=1 for AAPL:
Final Portfolio Value: $241,263.92
Total Return: 141.26%
Sharpe Ratio: 0.44
Max Drawdown: -58.06%

Testing weekly frequency strategy with k=1.5 for AAPL:
Final Portfolio Value: $170,399.68
Total Return: 70.40%
Sharpe Ratio: 0.32
Max Drawdown: -64.68%

Testing monthly frequency strategy with k=0.5 for AAPL:


C:\Users\Gautier Pellerin\Documents\Code\AMP_Algorithmic_Trading\strategies\..\strategies\ratio_value.py:57: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return data.resample(freq).last().dropna()


Final Portfolio Value: $252,202.44
Total Return: 152.20%
Sharpe Ratio: 0.45
Max Drawdown: -58.83%

Testing monthly frequency strategy with k=1 for AAPL:


...

Final Portfolio Value: $100,000.00
Total Return: 0.00%
Sharpe Ratio: nan
Max Drawdown: 0.00%

Testing 6month frequency strategy with k=1.5 for AEP:


C:\Users\Gautier Pellerin\Documents\Code\AMP_Algorithmic_Trading\strategies\..\strategies\ratio_value.py:57: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return data.resample(freq).last().dropna()


Final Portfolio Value: $100,000.00
Total Return: 0.00%
Sharpe Ratio: nan
Max Drawdown: 0.00%


C:\Users\Gautier Pellerin\AppData\Local\Temp\ipykernel_10944\2797192504.py:70: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  best_idx = comparison['Sharpe Ratio'].idxmax()


KeyError: nan

In [ ]:
# Get the top 2 most frequent (frequency, k) combinations
top_combinations = (
    best_params_df[['Best Frequency', 'Best k']]
    .value_counts()
    .head(2)
)

# Display them
print("\n✅ Top 2 most common parameter combinations across tickers:")
for i, ((freq, k), count) in enumerate(top_combinations.items(), start=1):
    print(f"{i}. Frequency: {freq}, k: {k} (appears {count} times)")
